# Performance by product --> SoM Dashboard

In [1]:
from pathlib import Path

BASE_DIR = Path('/Users/efraflores/Desktop/EF/Corner/Brands')
FILE_NAME = 'Nestle_marcas.csv'

## Import

In [2]:
import pandas as pd

df = pd.read_csv(BASE_DIR.joinpath(FILE_NAME))
df.columns = ['categoria','comp','nestle']
df.set_index('categoria', inplace=True)
df.sample(2)

,comp,nestle
categoria,,
Tés,"Davenport, Twinings, Alessa, La Pastora, Mc Co...",Nature's Heart
Chocolate Mesa,"Don GUSTAVO, Selecto; La Suiza Chocolate Mesa;...",Abuelita


## Transform

In [3]:
total = pd.DataFrame()

for col in df.columns:
    df[col] = df[col].str.replace(',',';')
    aux = df[col].str.split(';', expand=True).melt(ignore_index=False).dropna().iloc[:,-1:].rename({'value':'brand'},axis=1)
    aux['CPG'] = col.title()
    total = total.append(aux, ignore_index=False)

total['brand'] = total['brand'].str.strip()
total.sample(2)

,brand,CPG
categoria,,
Café Soluble,Juan Valdez,Comp
Culinarios,Maggi Inglesa,Nestle


In [4]:
import re

total['to_search'] = total['brand'].apply(lambda x: '%'+re.sub("[^a-z0-9]", '%', x.lower())+'%')
total['to_search'] = total['to_search'].apply(lambda x: "'"+re.sub("%{2}",'%',x)+"',")
total = total[total['brand']!=''].copy()
total.sample(4)

,brand,CPG,to_search
categoria,,,
Fórmulas Infantiles,NAN 1 y 2,Nestle,"'%nan%1%y%2%',"
Café Soluble,Genomma Lab,Comp,"'%genomma%lab%',"
Mascotas,Friskies,Nestle,"'%friskies%',"
Alimentos Infantiles,Buddy Fruits Original,Comp,"'%buddy%fruits%original%',"


## Export

In [5]:
total.to_csv(BASE_DIR.joinpath(''.join(FILE_NAME.split('.')[:-1])+'_correct.csv'), sep='\t', encoding='utf-16')